# Рекомендация тарифов

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Построить модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверить *accuracy* на тестовой выборке.

## Откроем и изучим файл

In [1]:
#импортируем необходимые библиотеки
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

In [2]:
#сохраним файл в df
df = pd.read_csv('https://code.s3.yandex.net/datasets/users_behavior.csv')

In [3]:
df

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


`calls` и `messages` можно перевести в int, a `is_ultra` в bool

### Вывод

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц.

Известно:

`сalls` — количество звонков,

`minutes` — суммарная длительность звонков в минутах,

`messages` — количество sms-сообщений,

`mb_used` — израсходованный интернет-трафик в Мб,

`is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).


## Разбьем данные на выборки

Определим переменные `features` (признаки) и `target` (целевой признак)

In [5]:
features = df.drop(['is_ultra'], axis=1)

In [6]:
target = df['is_ultra']

Разделим исходные данные на обучающую, валидационную и тестовую выборки `train`, `valid`, `test`, соответственно

In [7]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.2,train_size=0.8, random_state=2)
features_train, features_valid, target_train, target_valid = train_test_split(
    features_train, target_train,test_size = 0.25, train_size =0.75, random_state=2)

In [8]:
len(features_train)

1928

In [9]:
len(features_test)

643

In [10]:
len(features_valid)

643

In [11]:
features_valid

,calls,minutes,messages,mb_used
2357,120.0,932.14,6.0,35497.91
2914,57.0,428.23,5.0,8857.38
1015,42.0,319.80,121.0,16790.96
108,47.0,300.12,123.0,31244.61
2508,56.0,336.17,18.0,12500.44
...,...,...,...,...
1413,78.0,583.03,15.0,20350.96
1477,98.0,703.16,4.0,25391.67
2424,56.0,364.36,0.0,14129.94
2596,56.0,382.41,7.0,14563.80


### Вывод

Разделили исходные данные на обучающую, валидационную и тестовую выборки в соотношении 60%, 20%, 20%

## Обучение моделей

Обучим модель "Дерево решений"

In [12]:
model_tree = None
best_accuracy = 0
best_depth_tree = 0
for depth in range(1, 12):
    model = DecisionTreeClassifier(max_depth=depth, random_state=2) 
    model.fit(features_train, target_train) 
    predictions = model.predict(features_valid)
    accuracy_tree = accuracy_score(target_valid, predictions)
    if accuracy_tree > best_accuracy:
        best_accuracy = accuracy_tree
        best_depth_tree = depth
        model_tree = model
print("Accuracy наилучшей модели на валидационной выборке:", best_accuracy, "Глубина дерева:", best_depth_tree)

Accuracy наилучшей модели на валидационной выборке: 0.7916018662519441 Глубина дерева: 3


Обучим модель "Случайный лес"

In [13]:
model_forest = None
best_accuracy = 0
best_est = 0
best_depth_forest = 0
for est in range(1, 51, 5):
    for depth in range (1, 11):
        model = RandomForestClassifier(random_state=2, n_estimators=est)
        model.fit(features_train, target_train) 
        predictions = model.predict(features_valid)
        accuracy_forest = accuracy_score(target_valid, predictions)
        if accuracy_forest > best_accuracy:
            best_accuracy = accuracy_forest
            best_est = est
            best_depth_forest = depth
            model_forest = model
print("Accuracy наилучшей модели на валидационной выборке:", best_accuracy, "Количество деревьев:", best_est, "Максимальная глубина:", best_depth_forest)

Accuracy наилучшей модели на валидационной выборке: 0.7916018662519441 Количество деревьев: 6 Максимальная глубина: 1


Обучим модель "Логистическая регрессия" 

In [14]:
best_accuracy = 0
best_iter = 0
model_logistic = 0
for i in range(100, 1000, 20):
    model = LogisticRegression(random_state=42, max_iter=i) 
    model.fit(features_train, target_train) 
    predictions = model.predict(features_valid) 
    accuracy_logist = accuracy_score(target_valid, predictions)
    if accuracy_logist > best_accuracy:
        best_accuracy = accuracy_logist
        best_iter = i
        model_logistic = model
print("Accuracy наилучшей модели на валидационной выборке:", best_accuracy, "Число итераций:", best_iter)

Accuracy наилучшей модели на валидационной выборке: 0.7387247278382582 Число итераций: 100


### Вывод

На основе `train` данных были обучены три модели "Дерево решений", "Случайный лес" и "Логистическая регрессия".

На валидационной выборке получены следующие показатели точности (Accuracy):

Лучшую точность на валидационной выборке показала модель "Случайный лес" - Accuracy: 0.79, Количество деревьев: 35, Максимальная глубина: 1

Худшую точность показала модель "Логистическая регрессия" - Accuracy: 072, Число итераций: 100.

Между ними расположилась модель "Дерево решений" - Accuracy: 0.78, Глубина дерева: 4.

## Проверим модель на тестовой выборке

Проверим модель "Дерево решений" на тестовой выборке 

In [15]:
predictions = model_tree.predict(features_test)
accuracy_tree =  accuracy_score(target_test, predictions)
print("Accuracy на тестовой выборке:", accuracy_tree)

Accuracy на тестовой выборке: 0.7900466562986003


Проверим модель "Случайный лес" на тестовой выборке  

In [16]:
predictions = model_forest.predict(features_test)
accuracy_forest = accuracy_score(target_test, predictions)
print("Accuracy на тестовой выборке:", accuracy_forest)

Accuracy на тестовой выборке: 0.7978227060653188


Проверим модель "Логистическая регрессия" на тестовой выборке   

In [17]:
predictions = model_logistic.predict(features_test) 
accuracy_logist = accuracy_score(target_test, predictions)
print("Accuracy на тестовой выборке:", accuracy_logist)

Accuracy на тестовой выборке: 0.7418351477449455


### Вывод

На тестовой выборке получены следующие показатели точности (Accuracy):

Лучшую точность на тренировочной выборке показала модель "Случайный лес" - 0.80.

Худшую точность показала модель "Логистическая регрессия" - 070.

Между ними расположилась модель "Дерево решений" - 0.79.

## Проверим модели на адекватность

Проверим модели на адекватность, для этого используем классификатор Dummy

In [18]:
#проверим количество 0 и 1 в целевой выборке
target_test.value_counts()

0    437
1    206
Name: is_ultra, dtype: int64

Так как количество 0 в выборке сильно больше, чем 1, то используем классификатор Dummy со стратегией `prior` и заполним все предсказания нулями 

In [19]:
#используем классификатор Dummy со стратегией prior
dummy_prior = DummyClassifier(strategy="prior", random_state=42)
dummy_prior.fit(features_train, target_train)
predictions = dummy_prior.predict(features_test)
dummy_prior.score(target_test, predictions)
accuracy_dummy_prior = accuracy_score(target_test, predictions)
print("Accuracy при заполнении предсказания нулями:", accuracy_dummy_prior)

Accuracy при заполнении предсказания нулями: 0.6796267496111975


### Вывод 

1. Наилучшую точность показала модель "Случайный лес" - Accuracy = 0.80, но она и обучается дольше остальных.

2. На втором месте по точности модель "Дерево решений" - Accuracy = 0.79. Точность практически как у "Случайного леса", но обучается гораздо быстрее.

3. Худшую точность показала модель "Логистическая регрессия" - 0.70, она даже хуже, чем модель Dummy, заполняющая все предсказания нулями. Модель является не адекватной.

4. Оптимальной моделью является модель "Дерево решений" с глубиной 4.